# Benchmark HUC8 Streamflow Aggregation Queries

This notebook show how to query the NWM reanalysis dataset using HUC8s in various ways, time the queries, and save the results in a CSV.

## Setup

In [53]:
import json
from os.path import join
import time

from tqdm.notebook import tqdm
import geopandas as gpd
import xarray as xr
import fsspec
import numpy as np
import pyproj
from dask.distributed import Client
from dask_gateway import Gateway
import dask.dataframe as dd
import numpy as np
import pandas as pd
import fsspec

%matplotlib inline

def get_json(uri):
    with fsspec.open(uri) as fd:
        return json.load(fd)

In [76]:
# Connect to existing cluster using cluster.name

# This constant needs to be set!
cluster_name = 'daskhub.c1b31fa0a9d647779a036e43d0b42299'
gateway = Gateway()
cluster = gateway.connect(cluster_name)
client = cluster.get_client()

In [48]:
# Get COMIDs for a HUC8 around Philly from a HUC8 extract on S3.
# Each COMID represents a stream reach.

# Location of HUC8 extract JSON files.
huc8_root_uri = 's3://azavea-noaa-hydro-data/noaa/huc8-extracts/transformed/'

# TODO: run this with multiple HUC8s.
philly_huc8 = '02040202'
huc8_uri = join(huc8_root_uri, f'{philly_huc8}.json')

huc8_dict = get_json(huc8_uri)
comids = huc8_dict['features'][0]['properties']['comids']

# TODO: investigate the runtime warning below
ds = xr.open_zarr(zarr_uri)

# Apparently, only some of the reach ids in NHDPlus V2 are available in NWM.
# Question: why is that?
avail_comids = list(set(ds.feature_id.values).intersection(set(comids)))

# Need to sort or you will get warnings about a slowdown with an out of order index.
avail_comids.sort()
print(
    f'There are {len(comids)} reaches in the HUC and {len(avail_comids)} of those are in NWM.')

del ds

/tmp/ipykernel_139/270823347.py:15: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  ds = xr.open_zarr(zarr_uri)


There are 2153 reaches in the HUC and 1787 of those are in NWM.


## Run timing experiments

In [39]:
# Set things commons across all experiments.

# The number of times to repeat execution
nb_repeats = 5
time_ranges = [
    slice('1990-01-01', '2000-01-01'), 
    slice('1990-01-01', '1991-01-01'),
    slice('1990-01-01', '1990-02-01'),
]
nb_reaches = len(avail_comids)

### Zarr experiments

In [49]:
# Settings

# A map from query nicknames to functions that execute the query.
query_map = {
    'mean_features_mean_day': (lambda ds: ds.streamflow.mean(dim='feature_id').groupby('time.dayofyear').mean().values),
    'mean_day': (lambda ds: ds.streamflow.groupby('time.dayofyear').mean().values),
    'mean_week': (lambda ds: ds.streamflow.groupby('time.weekofyear').mean().values)
}
data_format = 'zarr'

# The CHRTOUT data from the NWM Retrospective Zarr 2.1 dataset
# This has "Streamflow values at points associated with flow lines" 
# See https://registry.opendata.aws/nwm-archive/
# Original dataset is at s3://noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr

# 10 year subset and transposed chunk version generated by save_zarr_data.ipynb
zarr_orig_uri = 's3://azavea-noaa-hydro-data/esip-experiments/datasets/reanalysis-chrtout/zarr/lf/07-07-2022c/nwm-subset.zarr'
zarr_trans_uri = 's3://azavea-noaa-hydro-data/esip-experiments/datasets/reanalysis-chrtout/zarr/lf/07-07-2022c/trans-chunk.zarr'
zarr_uris = [zarr_orig_uri, zarr_trans_uri]

zarr_results_uri = 's3://azavea-noaa-hydro-data/esip-experiments/benchmarks/zarr/lf/07-13-2022a.csv'

In [41]:
%%time

zarr_exp_rows = []
for zarr_uri in tqdm(zarr_uris, desc='zarr stores', leave=False):
    ds = xr.open_zarr(zarr_uri)
    for time_range in tqdm(time_ranges, desc='time ranges', leave=False):
        sub_ds = ds.sel(feature_id=avail_comids, time=time_range)

        nb_days = (pd.to_datetime(time_range.stop) - pd.to_datetime(time_range.start)).days
        
        chunk_sizes = np.array([ds.streamflow.chunks[0][0], ds.streamflow.chunks[1][0]])
        time_chunk_sz = chunk_sizes[0]
        feature_id_chunk_sz = chunk_sizes[1]
        
        for qname, qfunc in tqdm(query_map.items(), desc='query', leave=False):
            times = []
            for _ in tqdm(range(nb_repeats), desc='repeat', leave=False):
                start_time = time.time()
                vals = qfunc(sub_ds)
                elapsed = time.time() - start_time
                times.append(elapsed)
            times = np.array(times)
            exp_row = {
                'query': qname,
                'time_mean': times.mean(),
                'time_std': times.std(), 
                'nb_reaches': nb_reaches,
                'nb_days': nb_days,
                'nb_repeats': nb_repeats,
                'data_format': data_format,
                'time_chunk_sz': time_chunk_sz,
                'feature_id_chunk_sz': feature_id_chunk_sz
            }
            zarr_exp_rows.append(exp_row)
        del sub_ds
    del ds

zarr stores:   0%|          | 0/2 [00:00<?, ?it/s]

time ranges:   0%|          | 0/3 [00:00<?, ?it/s]

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


query:   0%|          | 0/3 [00:00<?, ?it/s]

repeat:   0%|          | 0/5 [00:00<?, ?it/s]

...

repeat:   0%|          | 0/5 [00:00<?, ?it/s]

repeat:   0%|          | 0/5 [00:00<?, ?it/s]

repeat:   0%|          | 0/5 [00:00<?, ?it/s]

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/accessor_dt.py:388: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/accessor_dt.py:388: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/accessor_dt.py:388: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/accessor_dt.py:388: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/accessor_dt.py:388: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalenda

CPU times: user 2min 8s, sys: 2.87 s, total: 2min 10s
Wall time: 12min 16s


In [46]:
exp_rows = zarr_exp_rows
df = pd.DataFrame(exp_rows)
df.to_csv(zarr_results_uri)
df

,query,time_mean,time_std,nb_reaches,nb_days,nb_repeats,data_format,time_chunk_sz,feature_id_chunk_sz
0,mean_features_mean_day,24.486602,2.643130,1787,3652,5,zarr,672,30000
1,mean_day,35.422201,1.138170,1787,3652,5,zarr,672,30000
2,mean_week,19.809512,0.144222,1787,3652,5,zarr,672,30000
3,mean_features_mean_day,3.427545,0.248162,1787,365,5,zarr,672,30000
4,mean_day,4.312051,0.088990,1787,365,5,zarr,672,30000
5,mean_week,2.948410,0.294242,1787,365,5,zarr,672,30000
6,mean_features_mean_day,0.922876,0.070954,1787,31,5,zarr,672,30000
7,mean_day,1.137972,0.263506,1787,31,5,zarr,672,30000
8,mean_week,0.870396,0.082696,1787,31,5,zarr,672,30000
9,mean_features_mean_day,5.304696,0.288083,1787,3652,5,zarr,30000,672


### Parquet experiments

In [96]:
# Settings

# A map from query nicknames to functions that execute the query.
query_map = {
    'mean_day': (lambda df: df.groupby(df.index.dayofyear).streamflow.mean().values.compute()),
    'mean_week': (lambda df: df.groupby(df.index.weekofyear).streamflow.mean().values.compute())
}
data_format = 'parquet'

# 10 years with metadata
parq_uri = 's3://azavea-noaa-hydro-data/esip-experiments/datasets/reanalysis-chrtout/parquet/vl/07-11-2022c-with-metadata/nwm-subset/'
parq_uris = [parq_uri]

parq_results_uri = 's3://azavea-noaa-hydro-data/esip-experiments/benchmarks/parquet/lf/07-13-2022a.csv'

In [97]:
# Run this block to configure a small set of experiments for testing purposes.
# This is useful since the Parquet experiments run so slowly currently.
nb_repeats = 1
query_map = {'mean_day': query_map['mean_day']}
time_ranges = [
    slice('1990-01-01', '1990-02-01'),
]

In [99]:
%%time

parq_exp_rows = []
for parq_uri in tqdm(parq_uris, desc='parqet stores', leave=False):
    df = dd.read_parquet(parq_uri, engine='pyarrow', index='time')
    for time_range in tqdm(time_ranges, desc='time ranges', leave=False):
        sub_df = df.query(
            'feature_id in @avail_comids and time > @start_time and time < @end_time', 
            local_dict={
                'avail_comids': avail_comids, 
                'start_time': time_range.start,
                'end_time': time_range.stop})

        nb_days = (pd.to_datetime(time_range.stop) - pd.to_datetime(time_range.start)).days
        
        # TODO: make the next lines of code valid
        time_chunk_sz = -9999
        feature_id_chunk_sz = -9999
        
        for qname, qfunc in tqdm(query_map.items(), desc='query', leave=False):
            times = []
            for _ in tqdm(range(nb_repeats), desc='repeat', leave=False):
                start_time = time.time()
                vals = qfunc(sub_df)
                elapsed = time.time() - start_time
                times.append(elapsed)
            times = np.array(times)
            exp_row = {
                'query': qname,
                'time_mean': times.mean(),
                'time_std': times.std(), 
                'nb_reaches': nb_reaches,
                'nb_days': nb_days,
                'nb_repeats': nb_repeats,
                'data_format': data_format,
                'time_chunk_sz': time_chunk_sz,
                'feature_id_chunk_sz': feature_id_chunk_sz
            }
            parq_exp_rows.append(exp_row)
        del sub_df
    del df

parqet stores:   0%|          | 0/1 [00:00<?, ?it/s]

time ranges:   0%|          | 0/1 [00:00<?, ?it/s]

query:   0%|          | 0/1 [00:00<?, ?it/s]

repeat:   0%|          | 0/1 [00:00<?, ?it/s]

Task exception was never retrieved
future: <Task finished name='Task-127704' coro=<Client._gather.<locals>.wait() done, defined at /srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/client.py:2006> exception=AllExit()>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/client.py", line 2015, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-127900' coro=<Client._gather.<locals>.wait() done, defined at /srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/client.py:2006> exception=AllExit()>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/client.py", line 2015, in wait
    raise AllExit()
distributed.client.AllExit


CPU times: user 2.32 s, sys: 67.5 ms, total: 2.38 s
Wall time: 4min 15s


In [100]:
df = pd.DataFrame(parq_exp_rows)
df.to_csv(parq_results_uri)
df

,query,time_mean,time_std,nb_reaches,nb_days,nb_repeats,data_format,time_chunk_sz,feature_id_chunk_sz
0,mean_day,255.483448,0.0,1787,31,1,parquet,-9999,-9999
